In [1]:
import numpy as np
import pandas as pd
import folium
import geocoder
from requests import Session

In [2]:
# numbeo = pd.read_csv('data/numbeo/numbeo.csv')
numbeo_clean = pd.read_csv('data/numbeo/numbeo_clean.csv')
numbeo_clean = numbeo_clean.drop(columns=['Unnamed: 0'])

In [3]:
numbeo_clean.sort_values('beer', ascending=False).head(25)

,city_ascii,admin_name,country,region,bread,beer,coffee,latitude,longitude,population
63,Doha,Ad Dawḩah,Qatar,Middle East,1.45,8.51,4.63,25.3000,51.5333,1312947.0
64,Amman,Al ‘Āşimah,Jordan,Middle East,0.41,5.30,4.14,31.9500,35.9333,4007526.0
65,Newcastle,New South Wales,Australia,Asia & Pacific,1.67,4.70,3.52,-32.9167,151.7500,322278.0
66,Muscat,Masqaţ,Oman,Middle East,1.08,4.66,4.61,23.6139,58.5922,1421409.0
67,Melbourne,Victoria,Australia,Asia & Pacific,2.34,4.57,3.40,-37.8136,144.9631,5078193.0
68,Adelaide,South Australia,Australia,Asia & Pacific,2.10,4.54,3.31,-34.9289,138.6011,1345777.0
69,Trondheim,Sør-Trøndelag,Norway,Europe,3.66,4.52,5.21,63.4400,10.4000,183378.0
70,Brisbane,Queensland,Australia,Asia & Pacific,2.12,4.50,3.55,-27.4678,153.0281,2514184.0
71,Canberra,Australian Capital Territory,Australia,Asia & Pacific,2.57,4.26,3.45,-35.2931,149.1269,426704.0
72,Sydney,New South Wales,Australia,Asia & Pacific,2.05,4.09,3.27,-33.8650,151.2094,5312163.0


In [4]:
import altair as alt

data = numbeo_clean.loc[numbeo_clean['city_ascii'] == "Seoul", ['beer', 'bread', 'coffee']]
d = data.transpose()
d = d.reset_index()
d = d.rename(columns={'index': "Seoul", 141: "price ($)"})

chart = alt.Chart(d).mark_bar().encode(x=d.columns[0], y=d.columns[1])

In [5]:
chart

alt.Chart(...)

In [6]:
chart.save("Seoul.json")
chart.save("Seoul.html")
c = chart.to_json()

In [7]:
import json

seoul_chart = json.loads(c)
popup = folium.Popup(max_width=450).add_child(folium.VegaLite(c))

In [81]:
import requests

url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
vis1 = json.loads(requests.get(f"{url}/vis1.json").text)
vis2 = json.loads(requests.get(f"{url}/vis2.json").text)
vis3 = json.loads(requests.get(f"{url}/vis3.json").text)

In [10]:
m = folium.Map(location=(numbeo_clean["latitude"][1], numbeo_clean["longitude"][1]), zoom_start=2)

fg = folium.FeatureGroup(name="markers").add_to(m)
m_data = zip(numbeo_clean['city_ascii'], zip(numbeo_clean['latitude'], numbeo_clean['longitude']), numbeo_clean["beer"])
for city, coord, price in m_data:
    fg.add_child(
        folium.CircleMarker(location=coord,
                            radius=3,
                            popup=city, 
                            tooltip=city, 
                            color="red", 
                            fill=True, 
                            fill_color="red"))

In [11]:
folium.LayerControl().add_to(m)
m